# SE-2200E Notebook 1: Data Collection

Ningsong Shen

December 14, 2020

## Summary

In this notebook, I describe the process by which we collect bus arrival time data for usage in the project. Simple scripts, a virtual machine, and one-click pipeline processing is used. Most of the work was done with scripts so this note book is mainly for description with some sample code.

## The Problem and Task Description

## Understanding Data Formats and API Usage

## Manual Experimentation and Data Transformation

## Automation with a Virtual Machine

## Data Storage and Transfer to Local Machine

## Larger Scale Processing and Automation

## Data Collection Timeline and Diminishing Returns